In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)

In [ ]:
plt.rcParams['font.size'] = 20
plt.rcParams['figure.titlesize'] = 20
plt.rcParams['figure.figsize'] = [9, 7]
plt.rcParams['font.family'] = ['SimHei']# ['Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus']=False 

In [ ]:
# Loading the dataset
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [ ]:
plt.imshow(X_test_orig[-2])

In [ ]:
print('Shape of X_train: ', X_train_orig.shape)
print('Shape of y_train: ', Y_train_orig.shape)
print('Shape of X_test: ', X_test_orig.shape)
print('Shape of y_test: ', Y_test_orig.shape)

In [ ]:
def preprocessing(X, y):
    # X:　(None, 64, 64, 3)
    X = tf.cast(X, dtype=tf.float32) / 255.
    X = tf.reshape(X, (-1, 64*64*3))
    y = tf.cast(y, dtype=tf.int32)
    y_onehot = tf.one_hot(y, depth=6)
    return X, y_onehot

In [ ]:
train_db = tf.data.Dataset.from_tensor_slices((X_train_orig, Y_train_orig))
test_db = tf.data.Dataset.from_tensor_slices((X_test_orig, Y_test_orig))
train_db = train_db.shuffle(2000).batch(64).map(preprocessing)
test_db = test_db.shuffle(2000).batch(64).map(preprocessing)

In [ ]:
import time

In [ ]:
losses = []
accuracy = []
max_epoch = 800
lr = 1e-3

# 初始化 参数
w1, b1 = tf.Variable(tf.random.normal([64*64*3, 25], stddev=0.1)), tf.Variable(tf.zeros([25]))
w2, b2 = tf.Variable(tf.random.normal([25, 12], stddev=0.1)), tf.Variable(tf.zeros([12]))
w3, b3 = tf.Variable(tf.random.normal([12, 6], stddev=0.1)), tf.Variable(tf.zeros([6]))

acc_meter = tf.keras.metrics.Accuracy()

optimizer = tf.keras.optimizers.Adam(0.0001)
st = time.process_time()
for epoch in range(max_epoch):
    cost = 0
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            h1 = x @ w1 + b1
            h1 = tf.nn.relu(h1)
            h2 = h1 @ w2 + b2
            h2 = tf.nn.relu(h2)
            out = h2 @ w3 + b3
            # out = tf.nn.softmax(out)
            loss = tf.keras.losses.categorical_crossentropy(y, out, from_logits=True)
            loss = tf.reduce_mean(loss)
            cost += loss
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        optimizer.apply_gradients(zip(grads, [w1, b1, w2, b2, w3, b3]))
#         for p, g in zip([w1, b1, w2, b2, w3, b3], grads):
#             p.assign_sub(lr * g)
    cost = cost / (step + 1)
            
    if epoch % 100 == 0:
        print(epoch, 'loss', float(cost))
        losses.append(cost)

        total_correct, nums = 0., X_test_orig.shape[0]
        for x, y in test_db:
            out = tf.nn.relu(tf.nn.relu(x@w1 + b1) @ w2 + b2) @ w3 + b3
            # h1 = x @ w1 + b1
            # h1 = tf.nn.relu(h1)
            # h2 = h1 @ w2 + b2
            # h2 = tf.nn.relu(h2)
            # out = h2 @ w3 + b3
            # 预测的标签
            y_pred = tf.argmax(out, axis=1)
            # 实际标签
            y_ = tf.argmax(y, axis=1)
            # 正确的
            acc_meter.update_state(y_, y_pred)
#             correct = tf.cast(y_pred == y_, dtype=tf.int32)
#             total_correct += tf.reduce_sum(correct).numpy()
        print(epoch, 'Evaluate Acc:', float(acc_meter.result()))
        accuracy.append(float(acc_meter.result()))
        acc_meter.reset_states()

end = time.process_time()
print('耗时:', end-st)
x = [i*100 for i in range(len(losses))]
plt.figure()
plt.plot(x, losses, color='C0', marker='s', label='训练误差')
plt.ylabel('Cross entropy')
plt.xlabel('epoch')
plt.legend()
plt.savefig('train.svg')

x = [i*100 for i in range(len(accuracy))]
plt.figure()
plt.plot(x, accuracy, color='C1', marker='s', label='测试集准确率')
plt.ylabel('准确率')
plt.xlabel('epoch')
plt.legend()
plt.savefig('test.svg')        

In [ ]:
image_file = r'datasets/fingers/2.png'
my_image = plt.imread(image_file)
out = tf.nn.relu(tf.nn.relu(my_image.reshape(1, -1)@w1 + b1) @ w2 + b2) @ w3 + b3
print("预测值: ", np.argmax(out))
plt.imshow(my_image)

In [ ]:
image_file = r'datasets/fingers/1.png'
my_image = plt.imread(image_file)
out = tf.nn.relu(tf.nn.relu(my_image.reshape(1, -1)@w1 + b1) @ w2 + b2) @ w3 + b3
print("预测值: ", np.argmax(out))
plt.imshow(my_image)

In [ ]:
image_file = r'datasets/fingers/3.png'
my_image = plt.imread(image_file)
out = tf.nn.relu(tf.nn.relu(my_image.reshape(1, -1)@w1 + b1) @ w2 + b2) @ w3 + b3
print("预测值: ", np.argmax(out))
plt.imshow(my_image)

In [ ]:
image_file = r'datasets/fingers/4.png'
my_image = plt.imread(image_file)
out = tf.nn.relu(tf.nn.relu(my_image.reshape(1, -1)@w1 + b1) @ w2 + b2) @ w3 + b3
print("预测值: ", np.argmax(out))
plt.imshow(my_image)

In [ ]:
image_file = r'datasets/fingers/5.png'
my_image = plt.imread(image_file)
out = tf.nn.relu(tf.nn.relu(my_image.reshape(1, -1)@w1 + b1) @ w2 + b2) @ w3 + b3
print("预测值: ", np.argmax(out))
plt.imshow(my_image)